<a href="https://colab.research.google.com/github/ganesh3/pytorch-work/blob/master/mnist_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
from torch import optim
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader

In [2]:
train_data = datasets.MNIST('data', train=True, download=True, transform = transforms.ToTensor())
train, val = random_split(train_data, [55000, 5000])
train_loader = DataLoader(train, batch_size = 32)
valid_loader = DataLoader(val, batch_size = 32)

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw


Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
#Define model
model = nn.Sequential(
    nn.Linear(28 * 28, 64),
    nn.ReLU(),
    nn.Linear(64, 64),
    nn.ReLU(),
    nn.Linear(64, 10)
)

In [13]:
getattr(model, 'parameters')

<bound method Module.parameters of Sequential(
  (0): Linear(in_features=784, out_features=64, bias=True)
  (1): ReLU()
  (2): Linear(in_features=64, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=10, bias=True)
)>

In [4]:
# Define optimizer
optimizer = optim.SGD(model.parameters(), lr=1e-2)

In [5]:
#Define loss
loss = nn.CrossEntropyLoss()

In [6]:
#training & validation loops
no_epochs = 5
for epoch in range(no_epochs):
  losses = list()
  accuracies = list()
  for batch in train_loader:
    x , y = batch

    #x: batchsize * 1 (no. of channel) * 28 (height) * 28 (width)
    b = x.size(0)
    x = x.view(b, -1)

    #1 forward
    l = model(x) # l = logits i.e. output form the last layer of the  model

    #2 compute objective function
    J = loss (l, y)

    #3 J cleaning the gradient
    model.zero_grad()
    #optimizer.zero_grad()
    #params.grad._zero()

    #4 accumulate the partial derivative of J w.r.t params
    J.backward()
    #params.grad.add_(dJ/dparams)

    #5 step in the opposite direction of the gradient
    optimizer.step()
    #with torch.no_grad: params = params - eta( i.e. learning-rate) * params.grad

    losses.append(J.item())
    accuracies.append(y.eq(l.detach().argmax(dim=1)).float().mean())

  print(f'Epoch {epoch + 1}, train loss: {torch.Tensor(losses).mean(): .2f}')
  print(f'Training accuracy: {torch.tensor(accuracies).mean(): .2f}')

  losses = list()
  accuracies = list()
  for batch in valid_loader:
    x , y = batch

    #x: batchsize * 1 (no. of channel) * 28 (height) * 28 (width)
    b = x.size(0)
    x = x.view(b, -1)

    #1 forward
    with torch.no_grad():
      l = model(x) # l = logits i.e. output form the last layer of the  model

    #2 compute objective function
    J = loss (l, y)

    losses.append(J.item())
    accuracies.append(y.eq(l.detach().argmax(dim=1)).float().mean())

  print(f'Epoch {epoch + 1}, validation loss: {torch.Tensor(losses).mean():.2f}')
  print(f'Validation accuracy: {torch.tensor(accuracies).mean(): .2f}')

Epoch 1, train loss:  1.26
Training accuracy:  0.65
Epoch 1, validation loss: 0.48
Validation accuracy:  0.86
Epoch 2, train loss:  0.40
Training accuracy:  0.89
Epoch 2, validation loss: 0.33
Validation accuracy:  0.90
Epoch 3, train loss:  0.32
Training accuracy:  0.91
Epoch 3, validation loss: 0.29
Validation accuracy:  0.92
Epoch 4, train loss:  0.28
Training accuracy:  0.92
Epoch 4, validation loss: 0.26
Validation accuracy:  0.92
Epoch 5, train loss:  0.25
Training accuracy:  0.93
Epoch 5, validation loss: 0.24
Validation accuracy:  0.93
